# setup

In [1]:
!pip install yt_dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import glob
import os
import pandas as pd
import yt_dlp

In [3]:
gdrive_home_path = "/content/drive"
from google.colab import drive
drive.mount(gdrive_home_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
DIR_NAME = "uraradi_transcript"

work_dir = os.path.join(gdrive_home_path, "MyDrive", DIR_NAME)
work_sounds_dir = os.path.join(work_dir, "sounds")
work_transcripts_dir = os.path.join(work_dir, "transcripts")

# identify existing mp3/transcript

In [5]:
mp3_list = []
for sounds_file in glob.glob(os.path.join(work_sounds_dir, "*.mp3")):
    filename = sounds_file.split("/")[-1].split(".")[0]
    mp3_list.append(filename)

csv_list = []
for transcripts_file in glob.glob(os.path.join(work_transcripts_dir, "*.csv")):
    filename = transcripts_file.split("/")[-1].split(".")[0]
    csv_list.append(filename)

# identify target radio

In [6]:
playlist = pd.read_csv(os.path.join(work_dir, "playlist_裏ラジオウルナイト.csv"))
playlist["flag"] = playlist["date"].apply(lambda date: (date not in mp3_list)&(date not in csv_list))
target_radios = playlist[playlist["flag"]==True][["url", "date"]].reset_index(drop=True).copy()
target_radios["finished"] = False
target_radios

,url,date,finished
0,https://www.youtube.com/watch?v=B1A8eGLymfo,2022-10-07,False
1,https://www.youtube.com/watch?v=aZ6hRj6jPFs,2022-07-08,False


# download mp3

- このセルを実行時にErrorが起きた際は、このセルのみを再実行してください。

In [7]:
ydl_opts = {
    "paths": {"home": work_sounds_dir},
    'format': 'm4a/bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
    }]
}
for _, radio in target_radios.iterrows():
    if target_radios.at[0, "finished"] == False:
        print(f"{radio.date} mp3 download start.")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download(radio["url"])
            info = ydl.extract_info(radio["url"], download=False)
        old_filename = info["title"].replace("/", "⧸") + " [" + info["id"] + "]" + ".mp3"
        new_filename = radio["date"] + ".mp3"
        if os.path.exists(os.path.join(work_sounds_dir, old_filename)):
            os.rename(os.path.join(work_sounds_dir, old_filename), os.path.join(work_sounds_dir, new_filename))
            print(f"{radio.date} mp3 download finish.")
        target_radios["finished"] == True

2022-10-07 mp3 download start.
[youtube] Extracting URL: https://www.youtube.com/watch?v=B1A8eGLymfo
[youtube] B1A8eGLymfo: Downloading webpage
[youtube] B1A8eGLymfo: Downloading android player API JSON
[youtube] B1A8eGLymfo: Downloading player 21246a91
[info] B1A8eGLymfo: Downloading 1 format(s): 140
[dashsegments] Total fragments: 10
[download] Destination: /content/drive/MyDrive/uraradi_transcript/sounds/【裏ラジ#61】ラインの向こう企画室 ⧸ 裏ラジオウルナイト【大浦るかこ ⧸ あにまーれ】 [B1A8eGLymfo].m4a
[download] 100% of   92.53MiB in 00:00:21 at 4.37MiB/s                 
[FixupM4a] Correcting container of "/content/drive/MyDrive/uraradi_transcript/sounds/【裏ラジ#61】ラインの向こう企画室 ⧸ 裏ラジオウルナイト【大浦るかこ ⧸ あにまーれ】 [B1A8eGLymfo].m4a"
[ExtractAudio] Destination: /content/drive/MyDrive/uraradi_transcript/sounds/【裏ラジ#61】ラインの向こう企画室 ⧸ 裏ラジオウルナイト【大浦るかこ ⧸ あにまーれ】 [B1A8eGLymfo].mp3
Deleting original file /content/drive/MyDrive/uraradi_transcript/sounds/【裏ラジ#61】ラインの向こう企画室 ⧸ 裏ラジオウルナイト【大浦るかこ ⧸ あにまーれ】 [B1A8eGLymfo].m4a (pass -k to keep)
[youtube